In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [86]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs=2
batch_size=4
learning_rate=0.001

transform=transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_dataset=torchvision.datasets.CIFAR10(root='./data',download=True,train=False,transform=transform)
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [87]:
classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [88]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1=nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,16*5*5)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
        
model=ConvNet().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

n_steps=len(train_loader)

for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        #change input shape
        images=images.to(device)
        labels=labels.to(device)
        
        #forwards
        outputs=model(images)
        loss=criterion(outputs,labels)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1)%2000==0:
            print(f'Epoch [{epoch+1}]/{num_epochs}, Step [{i+1}/{n_steps}], Loss:{loss.item():.4f}')

Epoch [1]/2, Step [2000/12500], Loss:2.2858
Epoch [1]/2, Step [4000/12500], Loss:2.2895
Epoch [1]/2, Step [6000/12500], Loss:2.3037
Epoch [1]/2, Step [8000/12500], Loss:2.3294
Epoch [1]/2, Step [10000/12500], Loss:2.2293
Epoch [1]/2, Step [12000/12500], Loss:2.2786
Epoch [2]/2, Step [2000/12500], Loss:3.0462
Epoch [2]/2, Step [4000/12500], Loss:2.2033
Epoch [2]/2, Step [6000/12500], Loss:1.8316
Epoch [2]/2, Step [8000/12500], Loss:2.5044
Epoch [2]/2, Step [10000/12500], Loss:1.7626
Epoch [2]/2, Step [12000/12500], Loss:2.1413


In [90]:
with torch.no_grad():
    n_correct=0
    n_samples=0
    n_class_correct=[0 for i in range(10)]
    n_class_samples=[0 for i in range (10)]
    
    for images,labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        
        _,predicted=torch.max(outputs,1)
        n_samples+=labels.size(0)
        n_correct+=(predicted==labels).sum().item()
        for i in range(batch_size):
            label=labels[i]
            pred=predicted[i]
            if (label==pred):
                n_class_correct[label]+=1
            n_class_samples[label]+=1

In [92]:
acc=100.0*n_correct/n_samples
acc

10.05

In [93]:
for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of plane: 26.7 %
Accuracy of car: 0.0 %
Accuracy of bird: 0.0 %
Accuracy of cat: 0.0 %
Accuracy of deer: 0.0 %
Accuracy of dog: 0.0 %
Accuracy of frog: 0.0 %
Accuracy of horse: 73.8 %
Accuracy of ship: 0.0 %
Accuracy of truck: 0.0 %
